# Découverte de mlflow, les cellule suivant sont basées sur les snippets générés par MLFLow UI, pour l'expérience "test"

In [8]:
from pathlib import Path

import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import mlflow
from packaging.version import Version

import mlflow.sklearn
from mlflow.entities import Dataset
from mlflow import set_tracking_uri

try:
    Path('mlruns').symlink_to('../mlruns')
except FileExistsError:
    assert Path('mlruns').readlink().samefile('../mlruns')

In [11]:

assert Version(mlflow.__version__) >= Version("2.14.3"), (
  "This feature requires MLflow version 2.14.3 or newer. "
  "Please run '%pip install -U mlflow' in a notebook cell, "
  "and restart the kernel when the command finishes."
)

@mlflow.trace
def foo(a): return a + bar(a)

# Various attributes can be passed to the decorator
# to modify the information contained in the span
@mlflow.trace(name = "custom_name", attributes = { "key": "value" })
def bar(b): return b + 1

# Invoking the traced function will cause a trace to be logged
foo(1)

3

In [10]:

mlflow.set_experiment(experiment_id="525382780087659060")

# Helper function to compute metrics
def compute_metrics(actual, predicted):
    rmse = mean_squared_error(actual, predicted)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)
    return rmse, mae, r2

# Load Iris dataset and prepare the DataFrame
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['quality'] = (iris.target == 2).astype(int)  # Create a binary target for simplicity

# Split into training and testing datasets
train_df, test_df = train_test_split(iris_df, test_size=0.2, random_state=42)

# Start a run to represent the training job
with mlflow.start_run() as training_run:
    # Load the training dataset with MLflow. We will link training metrics to this dataset.
    train_dataset: Dataset = mlflow.data.from_pandas(train_df, name="train")
    train_x = train_dataset.df.drop(["quality"], axis=1)
    train_y = train_dataset.df[["quality"]]

    # Fit a model to the training dataset
    lr = ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=42)
    lr.fit(train_x, train_y)

    # Log the model, specifying its ElasticNet parameters (alpha, l1_ratio)
    # As a new feature, the LoggedModel entity is linked to its name and params
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        name="elasticnet",
        params={
            "alpha": 0.5,
            "l1_ratio": 0.5,
        },
        input_example = train_x
    )

    # Inspect the LoggedModel and its properties
    logged_model = mlflow.get_logged_model(model_info.model_id)
    print(logged_model.model_id, logged_model.params)

    # Evaluate the model on the training dataset and log metrics
    # These metrics are now linked to the LoggedModel entity
    predictions = lr.predict(train_x)
    (rmse, mae, r2) = compute_metrics(train_y, predictions)
    mlflow.log_metrics(
        metrics={
            "rmse": rmse,
            "r2": r2,
            "mae": mae,
        },
        model_id=logged_model.model_id,
        dataset=train_dataset
    )

    # Inspect the LoggedModel, now with metrics
    logged_model = mlflow.get_logged_model(model_info.model_id)
    print(logged_model.model_id, logged_model.metrics)

m-d97b2e5f518e422480a68b81ec9c69ed {'alpha': '0.5', 'l1_ratio': '0.5'}
m-d97b2e5f518e422480a68b81ec9c69ed [<Metric: dataset_digest='d95ce242', dataset_name='train', key='rmse', model_id='m-d97b2e5f518e422480a68b81ec9c69ed', run_id='a945819386c54d369d99a747b05030a5', step=0, timestamp=1755434854280, value=0.1326400936461997>, <Metric: dataset_digest='d95ce242', dataset_name='train', key='mae', model_id='m-d97b2e5f518e422480a68b81ec9c69ed', run_id='a945819386c54d369d99a747b05030a5', step=0, timestamp=1755434854280, value=0.3209158276037644>, <Metric: dataset_digest='d95ce242', dataset_name='train', key='r2', model_id='m-d97b2e5f518e422480a68b81ec9c69ed', run_id='a945819386c54d369d99a747b05030a5', step=0, timestamp=1755434854280, value=0.3953727924959558>]


/home/nadir/Projects/p7.13.env/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
